In [135]:
# !pip install pyvi

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import re
from pyvi import ViTokenizer

In [137]:
# from google.colab import drive
# drive.mount('/content/drive')

In [138]:
# %cd /content/drive/My Drive/School/Năm 3/Data Analysis/Final
# %ls -la

In [4]:
data = pd.read_csv('./data/mobile_feedback/Train.csv')
data

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...
...,...,...,...,...,...
7781,7781,8g. Cái đi đánh là mạng giật giật ko chịu nổi....,1,13/10/2019,{FEATURES#Negative};{BATTERY#Negative};{PERFOR...
7782,7782,Mua dk giảm 500k mà lỗi lòi ra hết treo màn hì...,1,5/5/2020,{FEATURES#Negative};{PERFORMANCE#Negative};{PR...
7783,7783,Máy Sài 3 tháng rồi rất OK.pin trâu khỏi nói S...,5,23/12/2019,{BATTERY#Positive};{PERFORMANCE#Positive};{GEN...
7784,7784,"Rất tiếc hàng realme ko có ốp lưng ngoài , nên...",3,20/04/2020,{PRICE#Negative};{GENERAL#Positive};{SER&ACC#N...


In [140]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7786 entries, 0 to 7785
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      7786 non-null   int64 
 1   comment    7786 non-null   object
 2   n_star     7786 non-null   int64 
 3   date_time  7786 non-null   object
 4   label      7786 non-null   object
dtypes: int64(2), object(3)
memory usage: 304.3+ KB


## Xử lý ngôn ngữ tiếng Việt cho mỗi bình luận

In [141]:
def remove_pucntuation(comment):
  # Create a translation table
  translator = str.maketrans('', '', string.punctuation)
  # Remove punctuation
  new_string = comment.translate(translator)
  # Remove redudant space and break sign
  new_string = re.sub('[\n ]+', ' ', new_string)
  # Remove emoji icon
  emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags=re.UNICODE)
  new_string = re.sub(emoji_pattern, '', new_string)

  return new_string

In [142]:
def stopwords_vi():
    with open('./data/vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
        lines = file.readlines()
        words = [line.split('\n')[0] for line in lines]
    return words

In [143]:
def remove_stopword(comment):
  stop_words = stopwords_vi()
  filtered = [word for word in comment.split() if word not in stop_words]

  return ' '.join(filtered)

In [144]:
data['comment'] = data['comment'].apply(lambda x: x.lower())
data['comment'] = data['comment'].apply(remove_pucntuation)
data['comment'] = data['comment'].apply(lambda x: ViTokenizer.tokenize(x))
data['comment'] = data['comment'].apply(remove_stopword)

In [145]:
data['comment'][23]

'tất_cả mọi thứ ổn hết màn_hình đẹp máy chạy êm pin sạc nhanh màu mặt lưng đẹp tiết ở phần thiết_kế chi loa ngoài đặt ở bên hông thay_vì đặt ở cạnh tay đè loa sử_dụng í lo chính bản_thân đè quá làm loa hư thêm 1 nhỏ_xíu xiu mới mua hôm_qua giảm 500000đ nay xem tuột xuống đươc giảm hẳn 1000000đ tức ghê'

## Gán nhãn cho mỗi bình luận

In [146]:
data['positive_count'] = data['label'].apply(lambda x: x.count("Positive"))
data['neutral_count'] = data['label'].apply(lambda x: x.count("Neutral"))
data['negative_count'] = data['label'].apply(lambda x: x.count("Negative"))

In [147]:
def assign_label(row):
    if row['positive_count'] > row['neutral_count'] and row['positive_count'] > row['negative_count']:
        return 'Positive'
    elif row['negative_count'] > row['neutral_count'] and row['negative_count'] > row['positive_count']:
        return 'Negative'
    elif row['negative_count'] == row['neutral_count'] :
        return 'Negative'
    elif row['neutral_count'] == row ['positive_count']:
        return "Positive"
    else :
        return "Neutral"

In [148]:
data['label'] = data.apply(assign_label, axis=1)

In [149]:
data.head()

,index,comment,n_star,date_time,label,positive_count,neutral_count,negative_count
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,2 tuần trước,Positive,6,0,0
1,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,14/09/2019,Neutral,1,0,1
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,17/08/2020,Negative,0,0,1
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,29/02/2020,Neutral,0,2,1
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,4/6/2020,Positive,2,0,1


In [150]:
data['date_time'].value_counts()

date_time
2 tuần trước    216
3 tuần trước    197
1 tuần trước    169
4 tuần trước     70
21/04/2020       45
               ... 
15/06/2018        1
13/10/2018        1
22/07/2019        1
1/5/2018          1
15/06/2017        1
Name: count, Length: 756, dtype: int64

In [151]:
def replace_invalid_dates(temp):
    try:
        temp['date_time'] = pd.to_datetime(temp['date_time'], errors='coerce')
    except Exception as ex:
        pass

    temp['date_time'].fillna(value=temp['date_time'].mode()[0], inplace=True)
    
    return temp

data = replace_invalid_dates(data)

C:\Users\buidu\AppData\Local\Temp\ipykernel_13676\1929028042.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  temp['date_time'] = pd.to_datetime(temp['date_time'], errors='coerce')
C:\Users\buidu\AppData\Local\Temp\ipykernel_13676\1929028042.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  temp['date_time'].fillna(value=temp['date_time'].mode()[0], inplace=True)


In [152]:
data.head()

,index,comment,n_star,date_time,label,positive_count,neutral_count,negative_count
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,2020-04-21,Positive,6,0,0
1,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,2019-09-14,Neutral,1,0,1
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,2020-08-17,Negative,0,0,1
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,2020-02-29,Neutral,0,2,1
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,2020-04-06,Positive,2,0,1


In [153]:
print(data.shape)
print(data.columns)

(7786, 8)
Index(['index', 'comment', 'n_star', 'date_time', 'label', 'positive_count',
       'neutral_count', 'negative_count'],
      dtype='object')


In [154]:
data.head()

,index,comment,n_star,date_time,label,positive_count,neutral_count,negative_count
0,0,mới mua máy thegioididong thốt_nốt cảm_thấy ok...,5,2020-04-21,Positive,6,0,0
1,1,pin kém còn miễn chê mua 832019 tình_trạng pin...,5,2019-09-14,Neutral,1,0,1
2,2,sao gọi điện_thoại màn_hình chấm nhỏ nháy gần ...,3,2020-08-17,Negative,0,0,1
3,3,mọi người cập_nhật phần_mềm nó bớt tốn pin mìn...,3,2020-02-29,Neutral,0,2,1
4,4,mới mua sài 1 tháng thấy pin trâu sài bao mượt...,5,2020-04-06,Positive,2,0,1


In [155]:
data.to_csv("./data/mobile_feedback/Cleaned_Train.csv")